In [1]:
import sys
sys.path.append("../")  # path contains python_file.py

In [2]:
import iris

import numpy as np
import scipy.stats

In [3]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[1]

cm_hist = iris.load_cube(
    "testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc"
)
cm_future = iris.load_cube(
    "testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc"
)

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [4]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [5]:
# Get data
obs = obs.data[:, 0, :, :]
cm_hist = cm_hist.data
cm_future = cm_future.data

Test different debiasers:

In [7]:
from scipy.stats import norm

from PACKAGE_NAME.debias import QuantileMapping
from PACKAGE_NAME.debias import QuantileDeltaMapping
from PACKAGE_NAME.debias import ScaledDistributionMapping
from PACKAGE_NAME.debias import LinearScaling

distribution = norm

temp_debiaser = QuantileMapping(distribution=norm, delta_type="no_delta")
debiased_cm1 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileMapping(distribution=norm, delta_type="additive")
debiased_cm2 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = QuantileDeltaMapping(distribution=norm, time_window_length=50)
debiased_cm3 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = ScaledDistributionMapping()
debiased_cm4 = temp_debiaser.apply(obs, cm_hist, cm_future)

temp_debiaser = LinearScaling(delta_type = "additive")
debiased_cm5 = temp_debiaser.apply(obs, cm_hist, cm_future)

----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 952.41it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 820.71it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:16<00:00, 21.16it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:01<00:00, 348.44it/s]


----- Running debiasing -----


100%|██████████| 357/357 [00:00<00:00, 3732.84it/s]


In [8]:
%load_ext autoreload
%autoreload 2

In [10]:
from PACKAGE_NAME.debias import ScaledDistributionMapping

temp_debiaser = ScaledDistributionMapping()
debiased_cm4 = temp_debiaser.apply_location_temp(
    obs[:, 1, 1], cm_hist[:, 1, 1], cm_future[:, 1, 1]
)

In [12]:
from PACKAGE_NAME.debias import ScaledDistributionMapping

temp_debiaser = ScaledDistributionMapping()
debiased_cm5 = temp_debiaser.absolute_sdm_location(
    obs[:, 1, 1], cm_hist[:, 1, 1], cm_future[:, 1, 1]
)

In [13]:
debiased_cm4 - debiased_cm5

masked_array(data=[287.6973987569566, 346.57504309518794,
                   244.73950709169145, 219.84666240318074,
                   7.042065308558165, -54.438186385444396,
                   238.07054635983945, 229.40256067445318,
                   111.54022268357659, 104.42424531305471,
                   328.4870133939404, 300.7379878987197,
                   462.4124997645934, 333.21247811413673,
                   249.33243697791949, 248.39027934085323,
                   166.12964499658386, 189.92017628688848,
                   -140.5030112852436, -48.277592198232355,
                   -116.01724612880707, 38.1721654406698,
                   279.6453971863376, 228.78147376813183,
                   1253.9369644734925, 180.65364053992633,
                   243.5935646155155, 262.8294101807345,
                   307.8966824646343, -81.44355056240857,
                   -274.6104886695539, -317.3163050757439,
                   -22.935281669144388, 283.72632595416746,
    